In [11]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG

from IPython.display import Image, display
from sklearn.model_selection import train_test_split


In [8]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, N = read_VSP_label()

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [5]:
X_train, X_test, Y_train, Y_test = \
    split_material_validate(raw_feature, label_material, ratio=0.5)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(2622, 82)
(2623, 82)
(2622,)
(2623,)


In [10]:
label_visiprog = np.zeros((N,))

count = 0
for g in groups_VSP:
    label = False
    
    # check whether any entry in the group has been labelled yet
    # if yes, then merge with that existing label
    for i in g:
        if visiprog_label[i] != 0:
            label = label_visiprog[i]
            break
    
    # if no, then create a new label
    if label == False:
        count += 1
        label = count
        
    for i in g:
        label_visiprog[i] = label


In [20]:
accuracy = []

for i in range(10):
    X_train, X_test, Y_mat_train, Y_mat_test, Y_VSP_train, Y_VSP_test = \
    train_test_split(raw_feature, label_material, label_visiprog, \
                     test_size=0.5, stratify=label_material)


    model = LFDA_VISIPROG()
    model.fit(X_train, Y_VSP_train)

    X_vsp_test = model.transform(X_test)

    res = leave_one_sample_out(X_vsp_test, Y_mat_test)
    accuracy.append(res['accuracy'])

In [21]:
display(accuracy)

[0.96759435760579493,
 0.96950057186427752,
 0.96835684330918792,
 0.96454441479222264,
 0.96835684330918792,
 0.96797560045749143,
 0.96568814334731223,
 0.97255051467784981,
 0.96873808616088453,
 0.96911932901258102]

In [22]:
np.array(accuracy).mean()

0.96824247045367906